## インポート

In [9]:
from config import NUMBERS, NUMBERS3, NUMBERS4, NUMBERS3_RESULT, NUMBERS4_RESULT, NUMBERS3_VECTORS
import utils

import sys
import re
import difflib
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

plt.rcParams['font.family'] = 'AppleGothic'

name = "赤口"
type  = "+"

vec = utils.get_vectors(name, type, NUMBERS3_VECTORS)
print(vec)

%load_ext autoreload
%autoreload 2

[0, 2, 3]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### 調査 その1

In [3]:
X = []
Y = []
Z = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0
key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

"""
先勝・友引・先負・仏滅・大安・赤口
"""
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 2:
        if len(list(set(_y) & set(_v))) >= 2 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_time, _y, _v)
            # print(_format)
            pass
_sum     = sum([key_0, key_1, key_2, key_3, key_4, key_5])
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)

data = {
    "base": [key_0, key_1, key_2, key_3, key_4, key_5],
    "old_week": [key_6, key_7, key_8, key_9, key_10, key_11]
}
_data = pd.DataFrame(data)
print(_format1)
print(_format2)

# sns.countplot(x="old_week", data=_data)


前回:18個, 前々回:28個, 前回曜日:27個, 前々回曜日:18個, 前回六曜:23個, 前々回六曜:20個
赤口:0個, 仏滅:0個, 友引:0個 先勝:0個, 先負:134個, 大安:0個


In [29]:
# 4941
test = ["193", "413", "557", "648", "653", "669"] # 642 -> 464
# 0 -> 0
# 1 -> 2
# 2 -> 0
# 3 -> 2
# 4 -> 2
# 5 -> 3
# 6 -> 3
# 7 -> 1
# 8 -> 1
# 9 -> 2

# 4931
test = ["797", "513", "393", "909", "496", "934"] # 648 -> 480

# 4927 
test = ["393", "531", "851", "402", "934", "604"] # 496 -> 499

# 4919 
test = ["003", "402", "299", "002", "909", "372"] # 604 -> 466

"""
予測したものから、
(y, [x1, x2])
x1: 予測した数字
x2: 重複しなかった数字の合計
"""

'\n予測したものから、\n(y, [x1, x2])\nx1: 予測した数字\nx2: 重複しなかった数字の合計\n'

### 調査2

In [19]:
X = []
Y = []
Z = []
LOCK = {}
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    lock = []
    hook = []
    key_0 = 0
    key_1 = 0
    key_2 = 0
    key_3 = 0
    key_4 = 0
    key_5 = 0
    key_6 = 0
    key_7 = 0
    key_8 = 0
    key_9 = 0
    for _n in _x:
        if _time == _time:
            _h = list(_n)
            for _j in _h:
                if _j == '0':
                    key_0 += 1
                elif _j == '1':
                    key_1 += 1
                elif _j == '2':
                    key_2 += 1
                elif _j == '3':
                    key_3 += 1
                elif _j == '4':
                    key_4 += 1
                elif _j == '5':
                    key_5 += 1
                elif _j == '6':
                    key_6 += 1
                elif _j == '7':
                    key_7 += 1
                elif _j == '8':
                    key_8 += 1
                elif _j == '9':
                    key_9 += 1
            lock.append(list(_n))
    data = {
        "0": str(int(key_0) + 1),
        "1": str(int(key_1) + 1),
        "2": str(int(key_2) + 1),
        "3": str(int(key_3) + 1),
        "4": str(int(key_4) + 1),
        "5": str(int(key_5) + 1),
        "6": str(int(key_6) + 1),
        "7": str(int(key_7) + 1),
        "8": str(int(key_8) + 1),
        "9": str(int(key_9) + 1),
    }
    LOCK.update({ _time: [key_0, key_1, key_2, key_3, key_4, key_5, key_6, key_7, key_8, key_9]})
    Z.append(data)
    X.append(lock)
    
lock = pd.DataFrame(LOCK)

##### 調査3

In [ ]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS3_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x0 = NUMBERS3_RESULT[_i][_time][0]
    _x1 = NUMBERS3_RESULT[_i][_time][1]
    _x2 = NUMBERS3_RESULT[_i][_time][2]
    _x3 = NUMBERS3_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

### 設定

In [11]:
OLD_WEEK = "仏滅"
TYPE            = "+"

#### 特徴量

In [13]:
VEC = utils.get_vectors(OLD_WEEK, TYPE, NUMBERS3_VECTORS)

#### セット

In [14]:
X       = []
Y       = []
TIME = []
k1 = VEC[0]
k2 = VEC[1]
k3 = VEC[2]

#### 計算

In [ ]:
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][k1]
    _x2 = NUMBERS3[_i][_time][k2]
    _x3 = NUMBERS3[_i][_time][k3]
    _z = NUMBERS[_i][4]
    if not len(list(set(_y) & set(_x1))) >= 2 and not len(list(set(_y) & set(_x2))) >= 2 and _z == OLD_WEEK:
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass

### 予測スタート

###### 20180709

In [16]:
pkl_file = "data/numbers3_20180709.pkl"

###### + or -

In [17]:
data = [float("032"), float("487"), float("413")]

In [121]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [122]:
len(y)

171

In [123]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [124]:
px = x
py = model.predict(px)

In [125]:
NUM = []
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[2]) & set(_py))) >= 2:
                _format = "第{0}回: {1}<-{2}   予測:{3}".format(TIME[_i], _v[2], py[_i], LOCK[_v[0]])
                print(_format)
                NUM.append([TIME[_i], _v[2], py[_i], LOCK[_v[0]]])
                pass
            # print(TIME[_i], _v[2], py[_i])
print(len(NUM))

第4902回: 159<-591.8012834503295   予測:[1, 1, 1, 2, 2, 2, 2, 0, 2, 5]
第4889回: 684<-462.5220095125653   予測:[2, 1, 2, 1, 3, 2, 1, 1, 2, 3]
第4844回: 415<-475.50944647528183   予測:[0, 2, 5, 2, 0, 1, 1, 2, 4, 1]
第4811回: 937<-473.96745837548116   予測:[0, 4, 3, 0, 2, 1, 1, 3, 3, 1]
第4795回: 754<-437.21008431678223   予測:[2, 3, 2, 3, 0, 3, 2, 1, 2, 0]
第4766回: 656<-560.5214641586483   予測:[2, 0, 4, 1, 1, 2, 1, 4, 2, 1]
第4709回: 332<-432.120125286901   予測:[3, 2, 0, 1, 4, 1, 1, 1, 2, 3]
第4684回: 428<-423.92862273040373   予測:[2, 2, 3, 1, 1, 1, 1, 6, 1, 0]
第4635回: 984<-449.7684316603303   予測:[1, 1, 3, 2, 4, 1, 4, 1, 1, 0]
第4610回: 449<-479.5540156598464   予測:[3, 1, 1, 0, 3, 1, 1, 3, 4, 1]
第4567回: 415<-564.7920514606042   予測:[2, 3, 1, 2, 0, 2, 1, 3, 1, 3]
第4509回: 657<-467.88313142369174   予測:[0, 4, 0, 2, 1, 2, 5, 3, 1, 0]
第4396回: 942<-449.89973983474545   予測:[3, 2, 2, 2, 0, 0, 3, 2, 1, 3]
第4327回: 234<-542.0768861101469   予測:[1, 1, 2, 0, 2, 4, 1, 4, 2, 1]
第4310回: 988<-489.7980349521771   予測:[1, 1, 0, 1, 5, 2, 2,

## 決定係数

In [126]:
r2_score(y, py)

0.028018027827669045

In [127]:
model.score(x, y)

0.028018027827669045

In [128]:
model.coef_[0]

0.14132314515837072

In [131]:
joblib.dump(model, pkl_file)

['data/numbers3_20180709.pkl']

In [132]:
clf = joblib.load(pkl_file)

In [133]:
processor = clf.predict([data])

In [134]:
processor

array([449.05558779])